In [1]:
import numpy as np
import re
import json
import pandas as pd

In [24]:
labels = pd.read_csv("results.txt", delimiter = ",").loc[:,["pairid", " varA","varB","dataset","groundtruth"]]
labels['pairid'] = labels['pairid'].str.extract('(\d{4})').astype(int) # convert pairid to int
labels["groundtruth"] = labels["groundtruth"].str.replace(r'\s+', '', regex=True) # remove white space in labels

weights = pd.read_csv("pairmeta.txt", delimiter = " ", header = None).iloc[:,[0,5]]
weights.rename(columns = {0:"pairid", 5:"weight"}, inplace=True)
weights['pairid'] = weights['pairid'].astype(int) # convert pairid to int

merged_df = labels.merge(weights, on='pairid')
merged_df["gpt-4"] = ""

with open("gpt-4_response.json", "r") as file:
    loaded_data = json.load(file)
    
merged_df.head()

,pairid,varA,varB,dataset,groundtruth,weight,gpt-4
0,1,Altitude,Temperature,DWD,->,0.166,
1,2,Altitude,Precipitation,DWD,->,0.166,
2,3,Longitude,Temperature,DWD,->,0.167,
3,4,Altitude,Sunshine hours,DWD,->,0.166,
4,5,Age,Length,Abalone,->,0.143,


In [18]:
def add_prediction(merged_df, loaded_data, model):
    merged_df.set_index('pairid', inplace=True)
    for response in loaded_data:
        pairid, answer = extract_answers(str(response))
        print(pairid, answer)
        if answer != 'A/B' and answer[0] != 'E' and (answer[0]=='A' or answer[0]=='B'):
            answer = answer[0]
        answer = '->' if answer == 'A' else '<-' if answer == 'B' else answer
        merged_df.loc[int(pairid),model] = answer
        print("Converted:", answer, "\n")
    return merged_df


def extract_answers(s):
    # Compile regex patterns
    pairid_pattern = re.compile(r"\{'pair0(\d{3}),")
    answer_pattern = re.compile(r'<Answer>(.*?)</Answer>')
    
    # Search for pairid
    pairid_match = pairid_pattern.search(s)
    pairid = pairid_match.group(1) if pairid_match else None
    
    # Search for answer
    ret = answer_pattern.findall(s)
    
    # Error handling for answer
    if len(ret) > 1:
        return pairid, "Error: more than one"
    elif len(ret) == 0:
        return pairid, "Error: empty"
    
    return pairid, ret[0]

In [25]:
results = add_prediction(merged_df, loaded_data, "gpt-4")

001 A
Converted: -> 

002 A
Converted: -> 

003 A
Converted: -> 

004 A
Converted: -> 

005 A
Converted: -> 

006 A
Converted: -> 

007 A
Converted: -> 

008 A
Converted: -> 

009 A
Converted: -> 

010 A
Converted: -> 

011 A
Converted: -> 

012 A
Converted: -> 

013 A
Converted: -> 

014 A
Converted: -> 

015 A
Converted: -> 

016 A
Converted: -> 

017 A
Converted: -> 

018 A
Converted: -> 

019 A
Converted: -> 

020 A
Converted: -> 

021 A
Converted: -> 

022 A
Converted: -> 

023 A
Converted: -> 

024 A
Converted: -> 

025 A
Converted: -> 

026 A
Converted: -> 

027 A
Converted: -> 

028 A
Converted: -> 

029 A
Converted: -> 

030 A
Converted: -> 

031 A
Converted: -> 

032 A
Converted: -> 

033 A
Converted: -> 

034 A
Converted: -> 

035 A
Converted: -> 

036 A
Converted: -> 

037 A
Converted: -> 

038 A
Converted: -> 

039 A
Converted: -> 

040 A
Converted: -> 

041 A
Converted: -> 

042 A
Converted: -> 

043 A
Converted: -> 

044 A
Converted: -> 

045 A
Converted: -> 

046 A
Conv

In [22]:
results["correct"] = np.where(results["groundtruth"] == results["gpt-4"], 1, 0)
wtd_acc = sum(results["correct"]*results["weight"]) / results["weight"].sum()
acc = sum(results["correct"]) / 108
print("acc:", acc, "weighted acc:", wtd_acc)

acc: 0.9444444444444444 weighted acc: 0.9572464158326477
